In [2]:
pip install google-generativeai requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install google-genai pydantic

   ---------------------------------------- 0.0/703.4 kB ? eta -:--:--
   ---------------------------------------- 703.4/703.4 kB 3.1 MB/s  0:00:00

   ---------------------------------------- 0/8 [tenacity]
   ---------------------------------------- 0/8 [tenacity]
   ---------------------------------------- 0/8 [tenacity]
   ---------------------------------------- 0/8 [tenacity]
   ----- ---------------------------------- 1/8 [sniffio]
   ---------- ----------------------------- 2/8 [h11]
   ---------- ----------------------------- 2/8 [h11]
   ---------- ----------------------------- 2/8 [h11]
   ---------- ----------------------------- 2/8 [h11]
   --------------- ------------------------ 3/8 [distro]
   --------------- ------------------------ 3/8 [distro]
   -------------------- ------------------- 4/8 [anyio]
   -------------------- ------------------- 4/8 [anyio]
   -------------------- ------------------- 4/8 [anyio]
   -------------------- ------------------- 4/8 [anyio]
   


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from pydantic import BaseModel, Field

In [6]:
class TickertapeLink(BaseModel):
    """
    Defines the structure we expect from the AI model.
    """
    url: str = Field(description="The full URL to the stock's page on Tickertape.in")

In [40]:
# Correct way to read the Gemini API key from a file
with open(r"C:\Users\Rohit Jonnadula\secret_key.txt.txt", "r") as file:
    GEMINI_API_KEY = file.read().strip()

In [41]:
import google.generativeai as genai
import os

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY  # or set this in your environment
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

print("Gemini client initialized successfully.")

Gemini client initialized successfully.


In [42]:
from google.genai import types

def generate_link_with_gemini(ticker_symbol: str):
    # 3a. The prompt: The specific instruction for the AI.
    prompt = (
        f"Generate the exact Tickertape.in URL for the stock with ticker symbol '{ticker_symbol}'. "
        # f"The URL format is 'www.tickertape.in[TICKER_SYMBOL]'. "
        f"Only return the required JSON structure."
    )

    # 3b. The configuration: The rules for the output format.
    config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=TickertapeLink, # Points to our Pydantic class
    )
    
    # ... move to Step 4 ...
    print(f"Calling Gemini API for ticker: {ticker_symbol}...")
    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)

    # 5a. The response is a JSON string (e.g., '{"url": "..."}')
    print(f"Raw AI response text: {response.text}")

    # 5b. Use Pydantic to validate and parse the JSON string into our Python object
    data_object = TickertapeLink.model_validate_json(response.text)

    # 5c. Access the specific attribute (the link) and assign it to a Python variable
    final_link_variable = data_object.url

    return final_link_variable

In [43]:
ticker_to_search = "TCS" # Example for an Indian stock
my_final_url = generate_link_with_gemini(ticker_to_search)

print("-" * 40)
print(f"The final link assigned to the variable 'my_final_url' is:")
print(f"\n>>>> {my_final_url} <<<<\n")

Calling Gemini API for ticker: TCS...
Raw AI response text: ```json
{
  "url": "https://www.tickertape.in/stocks/tcs-TCS"
}
```
Raw AI response text: ```json
{
  "url": "https://www.tickertape.in/stocks/tcs-TCS"
}
```


ValidationError: 1 validation error for TickertapeLink
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "url": "ht...stocks/tcs-TCS"\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid

Error or unexpected response: {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}


In [11]:
import pandas as pd
import requests
import google.generativeai as genai
from bs4 import BeautifulSoup
ticker_name = 'jkil'
url = f'https://www.tickertape.in/stocks/{ticker_name}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify()[:1000])  # Print the first 1000 characters of the prettified HTML

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, viewport-fit=cover" name="viewport"/>
  <meta content="2" name="next-head-count"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="/images/favicon/favicon.png" rel="shortcut icon"/>
  <link href="/images/favicon/favicon-192x192.png" rel="apple-touch-icon"/>
  <link href="/manifest/manifest.json" rel="manifest"/>
  <style type="text/css">
   :root {--white: #ffffff; --font_primary: #535B62; --font_dark: #2f363f; --font_light: #81878c; --font_blue: #0088ea; --font_lighter: #a2a8ae; --brand_primary: #151e28; --brand_success: #28c39a; --brand_danger: #d93741; --brand_warn: #fbe283; --brand_bg: #f9f9f9; --brand_border: #d1d5d8; --brand_hover: #f7f7f7; --brand_gray: #acacac; --brand_light_gray: #e1e1e1; --purple_dark: #5311e9; --purple_light: #7656ff; --premium_dark: #624bff; --item_hover_bg: #373f48; --toast_err